<a href="https://colab.research.google.com/github/tusharsharm/Codsoft/blob/main/CUSTOMER_CHURN_PREDICTION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Import Datset

In [2]:
dataset = pd.read_csv("/content/Churn_Modelling.csv")

In [3]:
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
dataset = dataset.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

In [5]:
dataset.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


## Preprocessing

In [6]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

labelencoder_geography = LabelEncoder()
labelencoder_gender = LabelEncoder()

dataset['Geography'] = labelencoder_geography.fit_transform(dataset['Geography'])
dataset['Gender'] = labelencoder_gender.fit_transform(dataset['Gender'])

In [7]:
dataset.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,0,0,42,2,0.00,1,1,1,101348.88,1
1,608,2,0,41,1,83807.86,1,0,1,112542.58,0
2,502,0,0,42,8,159660.80,3,1,0,113931.57,1
3,699,0,0,39,1,0.00,2,0,0,93826.63,0
4,850,2,0,43,2,125510.82,1,1,1,79084.10,0


In [8]:
X = dataset.drop(columns=['Exited'])
y = dataset['Exited']

## Splitting into test and train dataset

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=0)

In [10]:
print(X_train.shape)
print(X_test.shape)

(8000, 10)
(2000, 10)


In [11]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Using Random Forest Classifier

In [25]:
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier()
classifier.fit(X_train, y_train)


RandomForestClassifier()

In [26]:
y_pred = classifier.predict(X_test)

In [27]:
print(y_pred)

[0 0 0 ... 0 0 0]


### Accuracy and Confusion matrix

In [28]:
from sklearn.metrics import accuracy_score,confusion_matrix
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
print("Accuracy:", accuracy)
print("\nConfusion Matrix:\n", conf_matrix)

Accuracy: 0.866

Confusion Matrix:
 [[1526   69]
 [ 199  206]]


## Fine tuning

In [30]:
from sklearn.model_selection import GridSearchCV

param_grid ={
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 4, 5],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5]
}

In [32]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# Define parameter grid for tuning
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Set up GridSearchCV with Random Forest
grid_search = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_grid=param_grid,
    scoring='accuracy',
    cv=5,
    n_jobs=-1,
    verbose=2
)

# Fit grid search
grid_search.fit(X_train, y_train)

# Best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best Accuracy:", grid_search.best_score_)


Fitting 5 folds for each of 81 candidates, totalling 405 fits
Best Parameters: {'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 300}
Best Accuracy: 0.8632500000000001


In [33]:
# Get the best model from GridSearchCV
best_classifier = grid_search.best_estimator_

# Predict on test data
y_pred_tuned = best_classifier.predict(X_test)

# Evaluate Performance (Optional)
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred_tuned)
print("Tuned Model Accuracy:", accuracy)


Tuned Model Accuracy: 0.864


In [34]:
print(y_pred_tuned)

[0 0 0 ... 0 0 0]


# Impoved accuracy after fine tuing

In [35]:
accuracy_tuned = accuracy_score(y_test, y_pred_tuned)
conf_matrix_tuned = confusion_matrix(y_test, y_pred_tuned)
print("\nImproved Accuracy after Tuning:", accuracy_tuned)
print("\nConfusion Matrix after Tuning:\n", conf_matrix_tuned)


Improved Accuracy after Tuning: 0.864

Confusion Matrix after Tuning:
 [[1527   68]
 [ 204  201]]
